# Fake News Classifier : Deep Learning

    Bidirectional RNN, LSTM, Dropout, Word Embedding, NLTK

    Link: https://www.kaggle.com/c/fake-news/data
    Dataset: 
        id: unique id for a news article
        title: the title of a news article
        author: author of the news article
        text: the text of the article; could be incomplete
        label: a label that marks the article as potentially unreliable
            1: unreliable
            0: reliable

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('fake-news/train.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
print(df.shape)

(20800, 5)


In [5]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [6]:
# Drop Nan Values
df = df.dropna()

In [7]:
X = df.drop('label',axis=1)
Y = df['label']

In [8]:
print(X.shape, Y.shape)

(18285, 4) (18285,)


In [9]:
import tensorflow as tf

In [10]:
tf.__version__

'2.7.0'

In [11]:
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.models import Sequential

In [12]:
# Vocabulary Size
voc_size = 5000

In [13]:
messages = X.copy()

In [14]:
messages['title'][1:3]

1    FLYNN: Hillary Clinton, Big Woman on Campus - ...
2                    Why the Truth Might Get You Fired
Name: title, dtype: object

In [15]:
# we have Dropped Nan values before so, reset_index() method sets a list of 
# integer ranging from 0 to length of data as index
messages.reset_index(inplace=True)

### Data Preprocessing 

In [16]:
import nltk 
import re
from nltk.corpus import stopwords

In [17]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sajwan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0,len(messages)):
    # print(i)
    review = re.sub('[^a-zA-z]',' ', messages['title'][i])
    # substituting everything with blank except alphabets
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)
# First for each index sentence, except all the alphabets all the words will be subsituted to ' '
# then conver each word into lowercase and split each word separately in the sentence.
# for each word not present in stopwords then only that word is stored in review
# again join the words that were splitted for forming a sentence and append together each sentence

In [19]:
corpus[:5]
# all the non necessary word is removed

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri']

### One Hot Representation

In [20]:
onehotRep = [one_hot(words, voc_size) for words in corpus]
print(onehotRep[:5])

[[1395, 1597, 1366, 96, 1515, 848, 1752, 2195, 1751, 1320], [1643, 1570, 2233, 1057, 3368, 3062, 3804], [3758, 213, 2294, 1978], [3744, 1098, 2329, 4108, 4590, 2035], [4454, 3368, 4629, 3654, 4037, 277, 3368, 2592, 4984, 1608]]


### Embedding Representation

In [21]:
# All the sent have variable len; using pad_sequence fix the len
sent_len = 20
embedded_docs = pad_sequences(onehotRep, padding = 'pre', maxlen = sent_len)
print(embedded_docs[:5])

[[   0    0    0    0    0    0    0    0    0    0 1395 1597 1366   96
  1515  848 1752 2195 1751 1320]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0 1643
  1570 2233 1057 3368 3062 3804]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0 3758  213 2294 1978]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
  3744 1098 2329 4108 4590 2035]
 [   0    0    0    0    0    0    0    0    0    0 4454 3368 4629 3654
  4037  277 3368 2592 4984 1608]]


In [22]:
len(embedded_docs)

18285

## Creating Model

In [23]:
from tensorflow.keras.layers import Bidirectional
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_len))
model.add(Bidirectional(LSTM(100))) 
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid')) # coz of classification prob
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 bidirectional (Bidirectiona  (None, 200)              112800    
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 200)               0         
                                                                 
 dense (Dense)               (None, 1)                 201       
                                                                 
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [24]:
import numpy as np
X_final = np.array(embedded_docs)
Y_final = np.array(Y)

In [25]:
print(X_final.shape, Y_final.shape)

(18285, 20) (18285,)


In [26]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X_final, Y_final, 
                                                test_size = 0.3, random_state=42)

## Model Training

In [27]:
model.fit(Xtrain,Ytrain,validation_data = (Xtest,Ytest), epochs=10, batch_size=64)

Epoch 1/10
200/200 [==============================] - 15s 52ms/step - loss: 0.2824 - accuracy: 0.8664 - val_loss: 0.2080 - val_accuracy: 0.9154
Epoch 2/10
200/200 [==============================] - 9s 46ms/step - loss: 0.1271 - accuracy: 0.9521 - val_loss: 0.2137 - val_accuracy: 0.9147
Epoch 3/10
200/200 [==============================] - 9s 46ms/step - loss: 0.0892 - accuracy: 0.9686 - val_loss: 0.2237 - val_accuracy: 0.9120
Epoch 4/10
200/200 [==============================] - 9s 46ms/step - loss: 0.0675 - accuracy: 0.9776 - val_loss: 0.3136 - val_accuracy: 0.9103: 0.0673 - 
Epoch 5/10
200/200 [==============================] - 9s 45ms/step - loss: 0.0509 - accuracy: 0.9845 - val_loss: 0.3698 - val_accuracy: 0.9118
Epoch 6/10
200/200 [==============================] - 9s 46ms/step - loss: 0.0390 - accuracy: 0.9871 - val_loss: 0.3706 - val_accuracy: 0.9096
Epoch 7/10
200/200 [==============================] - 9s 46ms/step - loss: 0.0293 - accuracy: 0.9909 - val_loss: 0.3760 - val_accu

In [28]:
# Since the accuracy is 0.9996 almost 1.0 it may lead to overfitting prob

## Performance Metrics and Accuracy

In [29]:
y_pred = model.predict_classes(Xtest)

AttributeError: 'Sequential' object has no attribute 'predict_classes'

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(Ytest,y_pred)
# False + and False - are very less compare to True + and True -

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(Ytest,y_pred)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(Ytest,y_pred))